In [ ]:
import os, sys

module_path = os.path.abspath(os.path.join('../../'))

if module_path not in sys.path:
    sys.path.append(module_path)

from src.data_fcts import *
from src.retrain_fcts import *

from src.bench_fcts import *
from src.viz_fcts import plot_detector_param_noise, plot_detector_noise_scenario, plot_detector_param_noise_scenario_specific
from ast import literal_eval

In [ ]:
all_noises = [0, 0.001, 0.01, 0.05, 0.1, 0.5]

In [ ]:

generate_nodrift_fcts = []
for noise_rate in all_noises:
    # NOISY 
    def generate_nodrift_df(noise_rate=noise_rate):

        D_G = Drift_generators(n_samples = n_samples, n_features = n_features)
        D_G.abrupt_concept_drift(drifts=[Drift(is_abrupt=True,
                                    start=n_samples-2,
                                    characteristic=[0.7-0.3*(0 % 2) for i in range(D_G.n_features)])])
        D_G.add_noise(noise_rate=noise_rate)
        D_G.drift_name = f"nodrift_noisy{str(noise_rate)[2:]}"
        
        return(D_G)
    generate_nodrift_fcts += [generate_nodrift_df]

In [ ]:
n_samples = 3000
n_features = 4

In [ ]:

generate_drift_fcts = []
for noise_rate in all_noises:
    # NOISY 
    def generate_noisy_smooth_concept_drift(n_samples = n_samples, noise_rate=noise_rate):
        D_G = Drift_generators(n_samples = n_samples, n_features = n_features)
        D_G.smooth_concept_drift(n_drift=1)
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)

    def generate_noisy_abrupt_concept_drift(n_samples = n_samples, noise_rate=noise_rate):
        D_G = Drift_generators(n_samples = n_samples, n_features = n_features)
        D_G.abrupt_concept_drift(n_drift=1)
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)

    def generate_noisy_gradual_concept_drift(n_samples = n_samples, noise_rate=noise_rate):
        D_G = Drift_generators(n_samples = n_samples, n_features = n_features)
        D_G.gradual_concept_drift(n_drift=1)
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)

    def generate_noisy_abrupt_covariate_drift(n_samples = n_samples, noise_rate=noise_rate):
        D_G = Drift_generators(n_samples = n_samples, n_features = n_features)
        d_centers = [(0, 0,
                    0.3, 0.25, 1),(0, 0.3, 0.5, 0, 0.25),(0, 0.5, 1, 0.25, 1)]
        D_G.abrupt_covariate_drift(d_centers=d_centers)
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)



            #generate_noisy_smooth_concept_drift
    noisy_fcts = [generate_noisy_smooth_concept_drift, generate_noisy_abrupt_concept_drift, generate_noisy_abrupt_covariate_drift,
                generate_noisy_gradual_concept_drift],
                 #generate_noisy_abrupt_covariate_drift]
                #generate_noisy_sine1, generate_noisy_sine2, generate_noisy_stagger]
    generate_drift_fcts += noisy_fcts


In [ ]:

list_df_generate_fcts_special = []
for noise_rate in all_noises:
    # NOISY 
    def generate_noisy_stagger(noise_rate=noise_rate):
        D_G = Drift_generators()
        D_G.load_df("stagger_short")
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)
#
    def generate_noisy_sine1(noise_rate=noise_rate):
        D_G = Drift_generators()
        D_G.load_df("sine1_short")
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)
#
    def generate_noisy_sine2(noise_rate=noise_rate):
        D_G = Drift_generators()
        D_G.load_df("sine2_short")
        D_G.add_noise(noise_rate=noise_rate)
        return(D_G)

    noisy_fcts = [generate_noisy_stagger, generate_noisy_sine1, generate_noisy_sine2],
    list_df_generate_fcts_special += noisy_fcts


In [ ]:
all_fcts_selected = [a+b for a,b in zip(list_df_generate_fcts_special, generate_drift_fcts)]
all_fcts_selected = [a+[b] for a,b in zip(all_fcts_selected, generate_nodrift_fcts)]

# Display all funcs

In [ ]:
all_fcts_selected[0]

# RUN results

In [ ]:
delta_d_list = [1, 0.8, 0.5, 0.25, 0.1,0.01, 0.001, 0.0001]
all_functions = all_fcts_selected
all_noise_rates = all_noises
df_perf_detector = bench_hyperparams_detector(
    "retrain_PH",
    "delta_d",
    delta_d_list,
    all_functions,
    all_noise_rates,
    save=True,
    path = os.environ.get("RESULTS_ROOT_PATH")+"detectors_param/PH_delta_d.csv"
)

In [ ]:
delta_d_list = [1, 0.8, 0.5, 0.25, 0.1,0.01, 0.001, 0.0001]
all_functions = all_fcts_selected
all_noise_rates = all_noises
df_perf_detector = bench_hyperparams_detector(
    "retrain_PH_shap",
    "delta_d",
    delta_d_list,
    all_functions,
    all_noise_rates,
    save=True,
    path = os.environ.get("RESULTS_ROOT_PATH")+"detectors_param/PH_shap_delta_d.csv"
)

# Display Results
## PH 

In [ ]:
root_path = os.environ.get("RESULTS_ROOT_PATH")+"detectors_param/"
name = "retrain_PH_delta_d.csv"
df_perf_detector = pd.read_csv(root_path+name).drop(columns = ['Unnamed: 0'])

df_perf_detector.loc[:,"drift_type"] = df_perf_detector.drift_name.apply(lambda name: "".join([x for x in name if not x.isdigit()]).replace("_noisy",""))
df_perf_detector = df_perf_detector.reset_index()

In [ ]:
param_name = "delta_d"
detector_name = "PH"

In [ ]:
df_perf_detector.groupby(["noise_rate", "delta_d"]).mean().head(3)

In [ ]:
ax = plot_detector_param_noise(df_perf_detector, param_name, detector_name, n_samples)

### Highlight scenarios which underperform

In [ ]:
ax = plot_detector_noise_scenario(df_perf_detector, param_name, detector_name, n_samples)

__Please select some param values__

In [ ]:
selected_delta_ds = [0.001]

In [ ]:
ax = plot_detector_param_noise_scenario_specific(df_perf_detector, param_name, detector_name, n_samples, selected_param_values=selected_delta_ds)

## Shap PH   

In [ ]:
root_path = os.environ.get("RESULTS_ROOT_PATH")+"detectors_param/"
name = "retrain_PH_shap_delta_d.csv"
df_perf_detector = pd.read_csv(root_path+name).drop(columns = ['Unnamed: 0'])

df_perf_detector.loc[:,"drift_type"] = df_perf_detector.drift_name.apply(lambda name: "".join([x for x in name if not x.isdigit()]).replace("_noisy",""))
df_perf_detector = df_perf_detector.reset_index()

In [ ]:
param_name = "delta_d"
detector_name = "SHAP PH"

In [ ]:
df_perf_detector.groupby(["noise_rate", "delta_d"]).mean().head(3)

In [ ]:
ax = plot_detector_param_noise(df_perf_detector, param_name, detector_name, n_samples)

### Highlight scenarios which underperform

In [ ]:
ax = plot_detector_noise_scenario(df_perf_detector, param_name, detector_name, n_samples)

__Please select some param values__

In [ ]:
selected_delta_ds = [0.1]


In [ ]:
ax = plot_detector_param_noise_scenario_specific(df_perf_detector, param_name, detector_name, n_samples, selected_param_values=selected_delta_ds)